<a href="https://colab.research.google.com/github/samina-if/AdventOfCode2024/blob/main/Advent_of_code(2025)Day7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import deque

def count_splits(grid):
    H = len(grid)
    W = len(grid[0])

    # Find S
    for r in range(H):
        for c in range(W):
            if grid[r][c] == 'S':
                sr, sc = r, c

    splits = 0
    used_splitters = set()
    visited_beams = set()    # <--- prevents infinite revisits

    q = deque()
    q.append((sr, sc))

    while q:
        r, c = q.popleft()

        # Avoid infinite loops
        if (r, c) in visited_beams:
            continue
        visited_beams.add((r, c))

        # Move downward
        rr = r
        while rr + 1 < H:
            rr += 1
            cell = grid[rr][c]

            if cell == '.':
                continue

            if cell == '^':
                # Count the splitter once
                if (rr, c) not in used_splitters:
                    used_splitters.add((rr, c))
                    splits += 1

                # Spawn new beams
                if c - 1 >= 0:
                    q.append((rr, c - 1))
                if c + 1 < W:
                    q.append((rr, c + 1))

                # Beam stops after hitting splitter
                break

            # If S or any other char acts like empty

    return splits


def read_input(path):
    with open(path, 'r') as f:
        lines = [line.rstrip('\n') for line in f]
    return [list(row) for row in lines]


if __name__ == "__main__":
    grid = read_input("input.txt")
    print(count_splits(grid))


In [ ]:
#!/usr/bin/env python3
from collections import deque, defaultdict
import sys

def read_input(path):
    with open(path, 'r', encoding='utf-8') as f:
        lines = [line.rstrip('\n') for line in f if line.rstrip('\n') != '']
    if not lines:
        raise ValueError("input is empty")
    maxw = max(len(l) for l in lines)
    grid = [list(l.ljust(maxw, '.')) for l in lines]
    return grid

def count_timelines(grid):
    H = len(grid)
    W = len(grid[0])

    # find S
    sr = sc = None
    for r in range(H):
        for c in range(W):
            if grid[r][c] == 'S':
                sr, sc = r, c
                break
        if sr is not None:
            break
    if sr is None:
        raise ValueError("No 'S' found in input")

    # counts[(r,c)] = number of particles starting at (r,c)
    counts = defaultdict(int)
    counts[(sr, sc)] = 1

    # queue of positions to process (we ensure each position is enqueued when it has pending count)
    q = deque([(sr, sc)])
    inqueue = set([(sr, sc)])

    finished = 0  # number of particles that exit bottom

    while q:
        r, c = q.popleft()
        inqueue.discard((r, c))
        n = counts.pop((r, c), 0)
        if n == 0:
            continue

        rr = r
        # move downward until exit or hit splitter
        while True:
            rr += 1
            if rr >= H:
                # exits manifold -> add to finished
                finished += n
                break
            cell = grid[rr][c]
            if cell == '.':
                continue
            if cell == '^':
                # splitter consumes these n particles and spawns n at left and n at right
                for nc in (c - 1, c + 1):
                    if 0 <= nc < W:
                        counts[(rr, nc)] += n
                        if (rr, nc) not in inqueue:
                            q.append((rr, nc))
                            inqueue.add((rr, nc))
                # beam stops at splitter
                break
            # treat 'S' or any other char as empty
            # continue downward

    return finished

if __name__ == "__main__":
    grid = read_input("input.txt")
    ans = count_timelines(grid)
    print(ans)
